# Demonstration: Upload metadata with impersonation

This takes a properly formatted metadata record with a linked file and uploads it to a user account using impersonation.

In [1]:
import json
import requests

In [11]:
#Set the token in the header and base URL

text_file = open("symp-figsh-token.txt", "r")
TOKEN = text_file.read()
TOKEN.strip() #removes any hidden spaces
text_file.close()


api_call_headers = {'Authorization': 'token ' + TOKEN}

account_id = 2023414 #account id for impersonation

#Set the base URL
BASE_URL = 'https://api.figsh.com/v2'

In [42]:
#Open a file
with open("beckham-record.json", "r", encoding='utf8') as read_file: #Replace this with the filename of your choice
    jsonfile = json.load(read_file)
    
#Add the impersonation value
jsonfile['impersonate'] = account_id

In [43]:
#Upload the records

record_fails = []

jsonresult = json.dumps(jsonfile)
r = requests.post(BASE_URL + '/account/articles', headers=api_call_headers, data = jsonresult)

if r.status_code != 201:
    record_fails.append(str(r.content[0:75])) #Add failed index to list with partial description
else:

    #Remove the admin account as an author by updating the record just created
    #This uses the article url returned by the API response (r)
    authordict = {}
    authordict['authors'] = jsonfile['authors']
    authordict['impersonate'] = account_id
    authorjson = json.dumps(authordict) #formats everything with double quotes
    #new_url = str(r.content)[39:-19] #Cut the response to just the URL
    response_json = json.loads(r.content)
    new_url = response_json['location']
    s = requests.put(new_url, headers=api_call_headers, data = authorjson) 
    
    #Upload a link as a file
    link = '{"link":"https://doi.org/10.1038/s41557-022-01030-y","impersonate":' + str(account_id) + '}' #hand send it for stage
    tup = new_url.rpartition('/') #From end of URL, split string '/' 
    article_id = str(tup[2]) #Then select the id from the resulting tuple.
    t = requests.post(BASE_URL + '/account/articles/' + str(article_id) +'/files', headers=api_call_headers, data = link)

    #Publish the record
    body = '{"impersonate":' + str(account_id) + '}'
    u = requests.post(BASE_URL + '/account/articles/' + str(article_id) +'/publish', headers=api_call_headers, data = body)
    
        
print('The record is created and available here:',json.loads(u.content)['location'])


0 record fails. Failed record indexes: []
